In [1]:
class Leaf:
    def __init__(self,left,right='$'):
        if len(left) == 0:
            self.left = '$'
            self.right = ''
        else:
            self.left = left
            self.right = right
    
    def __repr__(self): return '(' + repr(self.left) + ',' + repr(self.right) + ')'

    def __eq__(self,other): return self[0] == other[0]

    def __hash__(self): return hash(self[0])

    def __getitem__(self,index): return self.left[index]

    def __len__(self):
        if self.right == '$': return len(self.left) 
        return len(self.left)+len(self.right)
    
    # def __str_(self): return '\t' + str(self.left) + ':' + str(self.right) + '\n'

In [2]:
class Branch:
    def __init__(self):
        self.b = {}

    def __repr__(self): 
        s = ''
        for a in self.b.values(): 
            s += repr(a) 
        return s

    def __str__(self):
        s = '' 
        for i in range(len(list(self.b.values()))-1):s+=str(list(self.b.values())[i])+'\n'
        return s+str(list(self.b.values())[-1])
    
    def __getitem__(self,index): return self.b[index]
    
    def __contains__(self,other): return other in self.b

    def add(self,leaf):
        if leaf in self.b:
            if self.b[leaf].left == '$': return
            i = 0
            left = ''
            while i < min(len(leaf.left),len(self.b[leaf].left)) and leaf.left[i] == self.b[leaf][i]: 
                left += leaf.left[i]
                i += 1
            if type(self.b[leaf].right) == Branch:
                nleaf = Leaf(leaf.left[i:])
                if nleaf in self.b[leaf].right: self.b[leaf].right.add(nleaf)
            else: 
                branch = Branch()
                branch.add(Leaf(self.b[leaf].left[i:]))
                branch.add(Leaf(leaf.left[i:]))
                self.b[leaf].right = branch
            self.b[leaf].left = left
        else: self.b[leaf] = leaf

In [3]:
class Bonsai:
    def __init__(self):
        self.root = Branch()

    def add(self,text):
        for i in range(len(text)): self.root.sprout(Leaf(self.root,text[i:],'^$'))
        # text = ''.join(list(reversed(text)))
        # for i in range(len(text)): self.root.sprout(Leaf(self.root,text[i:],0))

In [7]:
b = Branch()
texts = ['hello','hollow','bellow']
for text in texts:
    for i in range(len(text)):
        b.add(Leaf(text[i:]))
print(b)

('h',('ello','$')('ollow','$'))
('ello',('$','')('w','$'))
('l',('lo',('$','')('w',('$','')))('o',('$','')('w',('$',''))))
('o',('$','')('llow','$'))
('w',('$',''))
('bellow','$')
